In [1]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 165kB/s 
     |████████████████████████████████| 983kB 42.8MB/s 
     |████████████████████████████████| 245kB 45.0MB/s 
     |████████████████████████████████| 1.1MB 41.6MB/s 
     |████████████████████████████████| 471kB 38.9MB/s 
     |████████████████████████████████| 3.0MB 35.6MB/s 
     |████████████████████████████████| 1.1MB 38.7MB/s 
     |████████████████████████████████| 890kB 40.8MB/s 
  Created wheel for ktrain: filename=ktrain-0.22.3-cp36-none-any.whl size=25271836 sha256=ec20ab43f99dae7e3ae4eb0e259cb472f01d19f063b197352f532eb04a20fb28
  Stored in directory: /root/.cache/pip/wheels/4a/4e/84/4a7f2f0a30761dc74ae9f3653f264c50837e7a8e7d08451354
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=bf4792a64a20f9063305dd41e011f6977ff377a8b2f2088179ca53e0ba8c48f8
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for lan

# Вопросно -ответные системы на пальцах

Потом на нейросетях вы поймёте детали, но там уже не будет NLP --- так что сейчас давайте поймём алгоритм.

Вопросно-ответные системы (QA, Open-Domain Question-Answering ) открытого домена принимают вопросы на естественном языке в качестве входных данных и возвращают точные ответы из больших текстовых корпусов, таких как Википедия (по сути, их задача высветить новое предложение).
 Это сильно отличается от стандартных поисковых систем, которые просто возвращают документы, соответствующие ключевым словам в поисковом запросе.




![picture](https://drive.google.com/uc?export=view&id=1LtA9PffMSJ7spAhWnsrr1YuhtUzwn8JU)

будем пользоваться нашим датасетом 20 newsgroups. Мы загрузим эти статьи и будем задавать вопросы по их содержанию, а оно нам должно будет дать верные ответы.

для этого нам понадобится ktrain

In [4]:
import ktrain
from ktrain import text

In [5]:
from sklearn.datasets import fetch_20newsgroups
remove = ('headers', 'footers', 'quotes')
newsgroups_train = fetch_20newsgroups(subset = 'train', remove = remove)
newsgroups_test = fetch_20newsgroups(subset = 'test', remove = remove)
docs = newsgroups_train.data + newsgroups_test.data

## Шаг 1. Search index

(Search engine optimisation indexing)

Нам надо быстр понять, в каких из присутсвующих документов есть слова из вопроса, бо если мы каждый раз будем бежать в цикле по каждому документу и смотрет, есть ли там это слово, это будет занимать время...

При создании поисковика мы работали быстрее - векторный поиск и матрица позволяющая понять в каких документах, какие слова, чтобы не анализировать .

(Document-term matrix) Это матрица, в которой по горизонтали слова, по вертикали - номера документов, а в местах их пересечения - frequency. 

Таблица - в первом столбце - слова, во втором - документы, в которых эти слова встречаются (Inverted matrix)

По второй быстрее искать, но нельзя сразу посмотреть частотность.

Индекс поиска позволит нам быстро и легко находить документы, содержащие слова, присутствующие в вопросе. Такие документы, скорее всего, содержат ответ и могут быть дополнительно проанализированы для извлечения возможных ответов. Сначала мы инициализируем поисковый индекс, а затем добавим в него документы из списка Python. 

Поскольку сообщения группы новостей небольшие и умещаются в основной памяти, мы установим для commit_every большое значение, чтобы ускорить процесс индексации.

In [6]:
INDEXDIR = '/tmp/myindex'

In [8]:
text.SimpleQA.initialize_index(INDEXDIR)


ValueError: ignored

In [9]:
text.SimpleQA.index_from_list(docs, INDEXDIR, commit_every=len(docs))

Для наборов документов, которые слишком велики для загрузки в список Python, вы можете использовать SimpleQA.index_from_folder, который будет сканировать папку и индексировать все найденные текстовые документы.

### Шаг 2. Create a QA Instance


Next, we will create a QA instance, which is largely a wrapper around a pretrained BertForQuestionAnswering model from the excellent transformers library.

Что же такое BERT? Объясняю простым языком.

https://www.youtube.com/watch?v=xI0HHN5XKDo


BERT зарадился из новой архитектуры для перевода слов.

Сейчас перевод делается через контекст --- чтобы посмотреть какое слово дальше вставить, мы должны посмотреть, какие слова были до и в завимости от этого вставить нужное слово.

LSTM 
1. медленные, так как не параллелят (в датасаенсе это важно!)
2. На самом деле, учат вообще только левый контекст, либо только правый, либо и тот, и тот по отдельности - а потом конкатинируют.


Потом пришли трансформеры.

Трансформер состоит из Encoder'a и Decoder'a. Encoder берет и сорздает вектора-эмбеддинги из всех слов в предложении СРАЗУ (БЫСТРО), учитывая ВЕСЬ КОНТЕКСТ ОДНОВРЕМЕННО.

(А потом Decoder берет уже последовательно переводит, он смотрит на первое слово - переводит его, потом смотрит на левый контекст -- смотрит всё, что было до него, переводим и т д., последовательно)

![picture](https://drive.google.com/uc?export=view&id=1N6S-nA9Zd5ZyXftGyjaPY4i6hbtcbJBB)


![picture](https://drive.google.com/uc?export=view&id=1kGMFD97-ac4Wv_ZxKx-2W9W-1WfoVqPr)



И вот если мы возьмём только верхний кусочек и немножко его модифицируем (стакнем три таких кусочка подряд) это и будет bert. И как говорит этот чувак в видео такая схема - это революционное приближение к тому, как МЫ понимаем язык, потому что она 1) вычленяет значения слов из широкого контекста, 2) смотрит не  только внутри предложения, но и за его пределами.


Мы тренируем BERT понимать язык (это уже сделано -- есть готовая натренированная на большом объеме данных модель от гугла), цель разработчков -- просто дальше "подкрутить BERT так, чтобы он решал именно их задачу.

Как именно тренировали Bert?

![picture](https://drive.google.com/uc?export=view&id=1m8sWLZbEo5yGOfbghhp31sDE1B0Cf0LK)




Есть 2 модели - большая и маленькая, которые очень хорошо понимают язык. Это уже сделано. Дальше вы их скачиваете и просто тьюните по вашу конкретную задачу. В этой библиотеке уже всё затьюнено под то, чтобы искать кусок, который отвечал бы на заданный вопрос (по сути, стоял бы вместо вопросительного слова). 

In [11]:
qa = text.SimpleQA(INDEXDIR)

qa = text. SimpleQA(INDEXDIR)

Мы вызовем метод qa.ask, чтобы выдавать вопросы в индексируемый нами текстовый корпус и получать ответы. Метод ask выполняет следующие шаги:
1. Использует поисковый индекс для поиска документов, содержащих слова в вопросе
2.Извлекает абзацы из этих документов  и использует модель BERT, предварительно обученную на наборе данных SQuAD, для анализа возможных ответов.
3. Сортирует и сокращает ответы кандидатов по шкале достоверности (confidence scores - https://www.aaai.org/Papers/Symposia/Spring/2003/SS-03-07/SS03-07-023.pdf) и возвращает результаты


Мы также будем использовать метод qa.display, чтобы красиво отформатировать и отобразить 5 лучших результатов в нашем ноутбуке. Поскольку модель просматривает абзацы и предложения в поисках ответов, для получения результатов может потребоваться некоторое время.

Также обратите внимание, что набор данных 20 групп новостей охватывает события с начала до середины 1990-х годов, поэтому отссылок к недавним событиям не будет. Однако набор данных охватывает множество различных категорий доменов. Например, есть категория sci.space, которая охватывает темы о космосе. Начнем с космического вопроса!

In [12]:
answers = qa.ask ('When did the Cassini probe launch?')
qa.display_answers(answers[:5])

,Candidate Answer,Context,Confidence,Document Reference
0,in october of 1997,cassini is scheduled for launch aboard a titan iv / centaur in october of 1997 .,0.348674,59
1,"on january 26,1962","ranger 3, launched on january 26,1962 , was intended to land an instrument capsule on the surface of the moon, but problems during the launch caused the probe to miss the moon and head into solar orbit.",0.195162,8525
2,"on november 5,1964","mariner 3, launched on november 5,1964 , was lost when its protective shroud failed to eject as the craft was placed into interplanetary space.",0.162834,8525
3,"launched october 18,1962","ranger 5, launched october 18,1962 and similar to ranger 3 and 4, lost all solar panel and battery power enroute and eventually missed the moon and drifted off into solar orbit.",0.077810,8525
4,2001,"possible launch dates : 1996 for imaging orbiter, 2001 for rover.",0.069740,59



![picture](https://drive.google.com/uc?export=view&id=1iN22nqn86vHUgIVqvZQpzzYD4kTgW6ht)

Bert используется для:


Neural machine Translation (GLUE score to 80.5% (7.7% point absolute improvement))


Question Answering (QuAD v1.1 question answering Test F1 to 93.2 (1.5 point absolute improvement))


Sentiment analysis


Text summarization